In [22]:
from __future__ import division, print_function
import sys
import os
import glob
import re
import numpy as np
from astropy.io import fits
from PIL import Image
from sklearn.metrics import accuracy_score
from flask import jsonify 
from io import BytesIO
import base64
import tensorflow as tf
import io

# Keras
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename 
from gevent.pywsgi import WSGIServer

In [23]:
# Define a flask app
app = Flask(__name__)

In [24]:
# Model saved with Keras model.save()
MODEL_PATH = "/Users/wendymendoza/Desktop/mycnn/CNNsingle.h5"

In [25]:
# Load your trained model
model = load_model(MODEL_PATH)
model.make_predict_function()          # Necessary
print('Model loaded. Start serving...')

Model loaded. Start serving...


In [26]:
@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')

In [27]:
@app.route('/predict', methods=['POST'])
def prediction_data():
    def resize_for_display(img, display_width=20):
        aspect_ratio = img.width / img.height
        display_height = int(display_width / aspect_ratio)
        display_img = img.resize((display_width, display_height), Image.BICUBIC)
        return np.asarray(display_img)

    images = []
    file = request.files['file']
    file_type = file.filename.split('.')[-1]
    if file_type == 'npy':
        data = np.load(file)
        print("Loaded .npy file, data shape:", data.shape, "data type:", data.dtype)
        num_images = data.shape[0]
        random_index = np.random.randint(0, num_images)
        data = data[random_index]

        # Create a PIL image from the numpy array for display purposes
        img = Image.fromarray((data * 255).astype(np.uint8))
        display_data = resize_for_display(img)

    elif file_type == 'fits':
        hdul = fits.open(file)
        data = hdul[0].data
        hdul.close()

        # Create a PIL image from the fits data for display purposes
        print("Creating image from data with shape:", data.shape)
        img = Image.fromarray((data * 255).astype(np.uint8))
        display_data = resize_for_display(img)

    elif file_type.lower() in ['jpg', 'jpeg', 'png']:
        img = Image.open(file)

        # For display purposes, resize the image while maintaining aspect ratio
        display_data = resize_for_display(img)

        # For the model prediction, resize the image to the required size
        model_img = img.resize((21, 21), Image.ANTIALIAS)
        data = np.asarray(model_img)
    else:
        return 'Error: Unsupported file type'
    
    original_image = display_data.copy()
    # preprocess data here, if necessary
    
    if len(data.shape) == 2:
        # Add a channel dimension to the data if it doesn't have one
        data = np.expand_dims(data, axis=-1)
    display_data = data.copy()
    data = data.astype(np.uint8)
    data = np.reshape(data, (-1, 21, 21, 2))
    data = data / 255.0  # normalize pixel values to [0, 1]
    print("Final data shape:", data.shape, "data type:", data.dtype)
    prediction = model.predict(data)
    # postprocess prediction here, if necessary
    prediction = np.argmax(prediction, axis=-1)

    if len(display_data.shape) == 4 and display_data.shape[-1] == 3:
        image = display_data[0, :, :, :]  # First RGB image
        
    elif len(display_data.shape) == 3:
        if display_data.shape[-1] == 3:
            image = display_data[:, :, :]
        else:
                image = display_data[:, :, 0]
    else:
        image = display_data[:, :, 0]

    # Convert the NumPy array to a PIL image and then to a base64 encoded string
    normalized_image = image / np.max(image)
    pil_image = Image.fromarray((normalized_image * 255).astype(np.uint8))
   
    buffer = BytesIO()
    pil_image.save(buffer, format="PNG")
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
    
    # Append the base64-encoded image string to the list of images
    images.append(image_base64) 

    #calculate predicition transient and non transient
    prediction = model.predict(data)
    prediction = np.argmax(prediction, axis=-1)
    prediction = np.where(prediction > 0.5, 1, 0)
    y_true = np.ones_like(prediction) #create an array of one with the same shape as precition
    accuracy = accuracy_score(y_true=y_true, y_pred=prediction)
    prediction = prediction.tolist()  # convert to list
    for i, p in enumerate(prediction):
        label = "transient" if p == 1 else "non-transient"
        print(f"File {i}: Prediction: {p}, Label: {label}")

    
    # return the prediction result and accuracy as a string
    return jsonify({"prediction": prediction, "accuracy": accuracy, "image_base64_list": images, "label": label})
    

In [29]:
if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


print("Creating image from data with shape:", data.shape)

print('Number of images in the loaded data:', data.shape[0])

print("Before resizing for display:")
print(np.asarray(img))


print("After resizing for display:")
print(display_data)


print("Before normalization:")
print(data)


print("After normalization:")
print(data)
